In [ ]:
#!/usr/bin/env python
# coding: utf-8


In [ ]:
# # Image Segmentation by Supervised Learning


In [ ]:
# ## import libraries


In [ ]:
# In[1]:


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import random
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

# ## load data


In [ ]:
# In[2]:


In [ ]:

directory_data  = './drive/MyDrive/Colab Notebooks/'
filename_data   = 'assignment_09_data.npz'
data            = np.load(os.path.join(directory_data, filename_data))


In [ ]:
x_train = data['x_train']
y_train = data['y_train']


In [ ]:
x_test  = data['x_test']
y_test  = data['y_test']


In [ ]:
num_data_train  = x_train.shape[0]
num_data_test   = x_test.shape[0]


In [ ]:
print('*************************************************')
print('size of x_train :', x_train.shape)
print('size of y_train :', y_train.shape)
print('*************************************************')
print('size of x_test :', x_test.shape)
print('size of y_test :', y_test.shape)
print('*************************************************')
print('number of training image :', x_train.shape[0])
print('height of training image :', x_train.shape[1])
print('width of training image :', x_train.shape[2])
print('*************************************************')
print('number of testing image :', x_test.shape[0])
print('height of testing image :', x_test.shape[1])
print('width of testing image :', x_test.shape[2])
print('*************************************************')


In [ ]:

# ## hyper-parameters


In [ ]:
# In[3]:


In [ ]:

device          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
number_epoch    = 1000
size_minibatch  = 20
learning_rate   = 0.05
weight_decay    = 0.0001


In [ ]:

# ## custom data loader for the PyTorch framework


In [ ]:
# In[13]:


In [ ]:

class dataset(Dataset):


In [ ]:
    def __init__(self, image, label, transform=False):


In [ ]:
        self.image  = image
        self.label  = label 
        self.transform = transform


In [ ]:
    def __getitem__(self, index):


In [ ]:
        image   = self.image[index]
        label   = self.label[index]


In [ ]:
        image   = torch.FloatTensor(image).unsqueeze(dim=0)
        label   = torch.FloatTensor(label).unsqueeze(dim=0)


In [ ]:
        image   = transforms.Resize(size=[64,64], interpolation=transforms.InterpolationMode('bilinear'))(image)
        label   = transforms.Resize(size=[64,64], interpolation=transforms.InterpolationMode('nearest'))(label)


In [ ]:
        if self.transform:
            # ==================================================
            # you can apply data augmentation here
            #


In [ ]:
            image = transforms.Compose([
                 transforms.RandomResizedCrop(24),
                 transforms.Pad(4, fill=0, padding_mode='constant')
                 ])(image)
            label = transforms.Compose(
                [
                 transforms.Resize(32),
                ])(label)


In [ ]:
            #
            # ==================================================
        return (image, label)


In [ ]:
    def __len__(self):


In [ ]:
        return self.image.shape[0]


In [ ]:

# ## construct datasets and dataloaders for training and testing


In [ ]:
# In[14]:


In [ ]:

dataset_train_transform = dataset(x_train, y_train, transform=True)
dataset_train           = dataset(x_train, y_train)
# WARNING: Multi Test Leakage — test data is being used improperly before final evaluation.
# QUICK FIX: Use K-Fold Cross Validation with n_splits=2 (based on 2 uses of test data).
dataset_test            = dataset(x_test, y_test)


In [ ]:
dataloader_train_transform  = DataLoader(dataset_train_transform, batch_size=size_minibatch, shuffle=True, drop_last=True)
dataloader_train            = DataLoader(dataset_train, batch_size=size_minibatch, shuffle=False, drop_last=True)
dataloader_test             = DataLoader(dataset_test, batch_size=size_minibatch, shuffle=False, drop_last=True)


In [ ]:

# ## shape of the data when using the data loader


In [ ]:
# In[15]:


In [ ]:

(image_train, label_train)  = dataset_train[0]
(image_test, label_test)    = dataset_test[0]
(image_train_transform, label_train_transform)  = dataset_train_transform[0]
print('*******************************************************************')
print('shape of the image in the training dataset:', image_train.shape)
print('shape of the label in the training dataset:', label_train.shape)
print('*******************************************************************')
print('shape of the image in the testing dataset:', image_test.shape)
print('shape of the label in the testing dataset:', label_test.shape)
print('*******************************************************************')
print('shape of the image in the training transform dataset:', image_train_transform.shape)
print('shape of the label in the training transform dataset:', label_train_transform.shape)
print('*******************************************************************')


In [ ]:

# ## class for the neural network 


In [ ]:
# In[17]:


In [ ]:

class Network(nn.Module): 


In [ ]:
	def __init__(self, in_channel=1, out_channel=1, dim_feature=128, dim_code=8, threshold_ReLU=0.001):


In [ ]:
		super(Network, self).__init__()


In [ ]:
		self.in_channel 	= in_channel
		self.out_channel	= out_channel
		self.dim_feature	= dim_feature


In [ ]:
		self.conv_encode1	= nn.Conv2d(in_channel , dim_feature * 1, kernel_size=3, stride=2, padding=1, bias=True)
		self.conv_encode2	= nn.Conv2d(dim_feature * 1, dim_feature * 2, kernel_size=3, stride=2, padding=1, bias=True)
		# self.conv_encode3	= nn.Conv2d(dim_feature * 2, dim_feature * 4, kernel_size=3, stride=2, padding=1, bias=True)
		# self.conv_encode4	= nn.Conv2d(dim_feature * 4, dim_feature * 8, kernel_size=3, stride=2, padding=1, bias=True)
		self.conv_middle	= nn.Conv2d(dim_feature * 2, dim_feature * 4, kernel_size=3, stride=1, padding=1, bias=True)
		# self.conv_decode4 	= nn.Conv2d(dim_feature * 16, dim_feature * 8, kernel_size=3, stride=1, padding=1, bias=True)
		# self.conv_decode3 	= nn.Conv2d(dim_feature * 8, dim_feature * 4, kernel_size=3, stride=1, padding=1, bias=True)
		self.conv_decode2 	= nn.Conv2d(dim_feature * 4, dim_feature * 2, kernel_size=3, stride=1, padding=1, bias=True)
		self.conv_decode1 	= nn.Conv2d(dim_feature * 2, dim_feature * 1,	kernel_size=3, stride=1, padding=1, bias=True)
		self.conv_out 		= nn.Conv2d(dim_feature * 1, out_channel,	kernel_size=1, stride=1, padding=0, bias=True)


In [ ]:
		self.ebn1			= nn.BatchNorm2d(dim_feature * 1)
		self.ebn2			= nn.BatchNorm2d(dim_feature * 2)
		# self.ebn3			= nn.BatchNorm2d(dim_feature * 4)
		# self.ebn4			= nn.BatchNorm2d(dim_feature * 8)
		self.mbn		 	= nn.BatchNorm2d(dim_feature * 4)
		# self.dbn4			= nn.BatchNorm2d(dim_feature * 8)
		# self.dbn3			= nn.BatchNorm2d(dim_feature * 4)
		self.dbn2			= nn.BatchNorm2d(dim_feature * 2)
		self.dbn1			= nn.BatchNorm2d(dim_feature * 1)


In [ ]:
		self.activation		= nn.ReLU(inplace=True)
		self.activation_out	= nn.Sigmoid()


In [ ]:
		# *********************************************************************
		# forward propagation
		# *********************************************************************
	def forward(self, x):


In [ ]:
		x1  = self.conv_encode1(x)
		eb1 = self.ebn1(x1)
		e1  = self.activation(eb1)


In [ ]:
		x2  = self.conv_encode2(e1)
		eb2 = self.ebn2(x2)
		e2  = self.activation(eb2)


In [ ]:
		# x3  = self.conv_encode3(e2)
		# eb3 = self.ebn3(x3)
		# e3  = self.activation(eb3)


In [ ]:
		# x4  = self.conv_encode4(e3)
		# eb4 = self.ebn4(x4)
		# e4  = self.activation(eb4)


In [ ]:
		m   = self.conv_middle(e2)
		mb  = self.mbn(m)
		c   = self.activation(mb)


In [ ]:
		# y4  = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)(c)
		# y4  = self.conv_decode4(y4) 
		# db4 = self.dbn4(y4)
		# d4  = self.activation(db4)


In [ ]:
		# y3  = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)(d4)
		# y3  = self.conv_decode3(y3) 
		# db3 = self.dbn3(y3)
		# d3  = self.activation(db3)


In [ ]:
		y2  = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)(c)
		y2  = self.conv_decode2(y2) 
		db2 = self.dbn2(y2)
		d2  = self.activation(db2)


In [ ]:
		y1  = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)(d2)
		y1  = self.conv_decode1(y1) 
		db1 = self.dbn1(y1)
		d1  = self.activation(db1)


In [ ]:
		y1  = self.conv_out(d1)
		y = self.activation_out(y1)


In [ ]:
		return y


In [ ]:

# ## build network


In [ ]:
# In[18]:


In [ ]:

model       = Network().to(device)
optimizer   = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


In [ ]:

# ## compute the prediction


In [ ]:
# In[19]:


In [ ]:

def compute_prediction(model, input):
    # ==================================================
    # fill up the blank
    #


In [ ]:
    prediction = model(input)


In [ ]:
    # 
    # ==================================================
    return prediction


In [ ]:

# ## compute the loss


In [ ]:
# In[20]:


In [ ]:

def compute_loss(prediction, label):
    # ==================================================
    # fill up the blank
    #
    mse = nn.MSELoss()
    loss =  mse(prediction, label)


In [ ]:
    # 
    # ==================================================
    loss_value  = loss.item()


In [ ]:
    return loss, loss_value


In [ ]:

# ## compute the accuracy


In [ ]:
# In[21]:


In [ ]:

def compute_accuracy(prediction, label):


In [ ]:
    prediction  = prediction.squeeze(axis=1)
    label       = label.squeeze(axis=1)


In [ ]:
    prediction_binary   = (prediction >= 0.5).cpu().numpy()
    label               = label.bool().cpu().numpy()


In [ ]:
    region_intersection = prediction_binary & label
    region_union        = prediction_binary | label


In [ ]:
    area_intersection   = region_intersection.sum(axis=1).sum(axis=1).astype(float)
    area_union          = region_union.sum(axis=1).sum(axis=1).astype(float)


In [ ]:
    eps         = np.finfo(float).eps
    correct     = area_intersection / (area_union + eps)
    accuracy    = correct.mean() * 100.0


In [ ]:
    return accuracy


In [ ]:

# ## variables for the learning curve


In [ ]:
# In[22]:


In [ ]:

loss_mean_train     = np.zeros(number_epoch)
loss_std_train      = np.zeros(number_epoch)
accuracy_mean_train = np.zeros(number_epoch)
accuracy_std_train  = np.zeros(number_epoch)


In [ ]:
loss_mean_test      = np.zeros(number_epoch)
loss_std_test       = np.zeros(number_epoch)
accuracy_mean_test  = np.zeros(number_epoch)
accuracy_std_test   = np.zeros(number_epoch)


In [ ]:

# ## train


In [ ]:
# In[23]:


In [ ]:

def train(model, dataloader):


In [ ]:
    loss_epoch      = []
    accuracy_epoch  = []


In [ ]:
    model.train()


In [ ]:
    for index_batch, (image, label) in enumerate(dataloader):


In [ ]:
        image = image.to(device)
        label = label.to(device)


In [ ]:
        # ==================================================
        # fill up the blank 
        #


In [ ]:
        prediction          = compute_prediction(model, image)
        loss, loss_value    = compute_loss(prediction, label)
        accuracy            = compute_accuracy(prediction, label)


In [ ]:
        # 
        # ==================================================


In [ ]:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
        loss_epoch.append(loss_value)
        accuracy_epoch.append(accuracy)


In [ ]:
    loss_mean_epoch     = np.mean(loss_epoch)
    loss_std_epoch      = np.std(loss_epoch)


In [ ]:
    accuracy_mean_epoch = np.mean(accuracy_epoch)
    accuracy_std_epoch  = np.std(accuracy_epoch)


In [ ]:
    loss        = {'mean' : loss_mean_epoch, 'std' : loss_std_epoch}
    accuracy    = {'mean' : accuracy_mean_epoch, 'std' : accuracy_std_epoch}


In [ ]:
    return (loss, accuracy)    


In [ ]:

# ## test


In [ ]:
# In[24]:


In [ ]:

def test(model, dataloader):


In [ ]:
    loss_epoch      = []
    accuracy_epoch  = []


In [ ]:
    model.eval()


In [ ]:
    for index_batch, (image, label) in enumerate(dataloader):


In [ ]:
        image = image.to(device)
        label = label.to(device)


In [ ]:
        # ==================================================
        # fill up the blank 
        #


In [ ]:
        prediction          = compute_prediction(model, image)
        loss, loss_value    = compute_loss(prediction, label)
        accuracy            = compute_accuracy(prediction, label)


In [ ]:
        # 
        # ==================================================


In [ ]:
        loss_epoch.append(loss_value)
        accuracy_epoch.append(accuracy)


In [ ]:
    loss_mean_epoch     = np.mean(loss_epoch)
    loss_std_epoch      = np.std(loss_epoch)


In [ ]:
    accuracy_mean_epoch = np.mean(accuracy_epoch)
    accuracy_std_epoch  = np.std(accuracy_epoch)


In [ ]:
    loss        = {'mean' : loss_mean_epoch, 'std' : loss_std_epoch}
    accuracy    = {'mean' : accuracy_mean_epoch, 'std' : accuracy_std_epoch}


In [ ]:
    return (loss, accuracy)    


In [ ]:

# ## train and test


In [ ]:
# In[25]:


In [ ]:

# ================================================================================
# 
# iterations for epochs
#
# ================================================================================
for i in tqdm(range(number_epoch)):


In [ ]:
    # ================================================================================
    # 
    # training
    #
    # ================================================================================
    (loss_train, accuracy_train) = train(model, dataloader_train_transform)


In [ ]:
    loss_mean_train[i]      = loss_train['mean']
    loss_std_train[i]       = loss_train['std']


In [ ]:
    accuracy_mean_train[i]  = accuracy_train['mean']
    accuracy_std_train[i]   = accuracy_train['std']


In [ ]:
    # ================================================================================
    # 
    # testing
    #
    # ================================================================================
    (loss_test, accuracy_test) = test(model, dataloader_test)


In [ ]:
    loss_mean_test[i]      = loss_test['mean']
    loss_std_test[i]       = loss_test['std']


In [ ]:
    accuracy_mean_test[i]  = accuracy_test['mean']
    accuracy_std_test[i]   = accuracy_test['std']


In [ ]:

# ---


In [ ]:
# # functions for visualizing the results 


In [ ]:
# ---


In [ ]:
# ## plot curve


In [ ]:
# 


In [ ]:
# In[26]:


In [ ]:

def plot_data_grid(data, index_data, nRow, nCol):


In [ ]:
    size_col = 1.5
    size_row = 1.5


In [ ]:
    fig, axes = plt.subplots(nRow, nCol, constrained_layout=True, figsize=(nCol * size_col, nRow * size_row))


In [ ]:
    for i in range(nRow):
        for j in range(nCol):


In [ ]:
            k       = i * nCol + j
            index   = index_data[k]


In [ ]:
            axes[i, j].imshow(data[index], cmap='gray', vmin=0, vmax=1)
            axes[i, j].xaxis.set_visible(False)
            axes[i, j].yaxis.set_visible(False)


In [ ]:
    plt.show()


In [ ]:

# In[27]:


In [ ]:

def plot_data_tensor_grid(data, index_data, nRow, nCol):


In [ ]:
    size_col = 1.5
    size_row = 1.5


In [ ]:
    fig, axes = plt.subplots(nRow, nCol, constrained_layout=True, figsize=(nCol * size_col, nRow * size_row))


In [ ]:
    data = data.detach().cpu().squeeze(axis=1)


In [ ]:
    for i in range(nRow):
        for j in range(nCol):


In [ ]:
            k       = i * nCol + j
            index   = index_data[k]


In [ ]:
            axes[i, j].imshow(data[index], cmap='gray', vmin=0, vmax=1)
            axes[i, j].xaxis.set_visible(False)
            axes[i, j].yaxis.set_visible(False)


In [ ]:
    plt.show()


In [ ]:

# In[28]:


In [ ]:

def plot_curve_error(data_mean, data_std, x_label, y_label, title):


In [ ]:
    plt.figure(figsize=(8, 6))
    plt.title(title)


In [ ]:
    alpha = 0.3


In [ ]:
    plt.plot(range(len(data_mean)), data_mean, '-', color = 'red')
    plt.fill_between(range(len(data_mean)), data_mean - data_std, data_mean + data_std, facecolor = 'blue', alpha = alpha) 


In [ ]:
    plt.xlabel(x_label)
    plt.ylabel(y_label)


In [ ]:
    plt.tight_layout()
    plt.show()


In [ ]:

# In[29]:


In [ ]:

def print_curve(data, index):


In [ ]:
    for i in range(len(index)):


In [ ]:
        idx = index[i]
        val = data[idx]


In [ ]:
        print('index = %2d, value = %12.10f' % (idx, val))


In [ ]:

# In[30]:


In [ ]:

def get_data_last(data, index_start):


In [ ]:
    data_last = data[index_start:]


In [ ]:
    return data_last


In [ ]:

# In[31]:


In [ ]:

def get_max_last_range(data, index_start):


In [ ]:
    data_range = get_data_last(data, index_start)
    value = data_range.max()


In [ ]:
    return value


In [ ]:

# In[32]:


In [ ]:

def get_min_last_range(data, index_start):


In [ ]:
    data_range = get_data_last(data, index_start)
    value = data_range.min()


In [ ]:
    return value


In [ ]:

# ---


In [ ]:
# # functions for presenting the results


In [ ]:
# ---


In [ ]:
# In[33]:


In [ ]:

def function_result_01():


In [ ]:
    print('[plot examples of the training images]')
    print('') 


In [ ]:
    nRow = 8
    nCol = 6
    index_data  = np.arange(0, nRow * nCol)
    image_train,_ = dataset_train[index_data]
    image_train = image_train[0]


In [ ]:
    plot_data_grid(image_train, index_data, nRow, nCol)


In [ ]:

# In[34]:


In [ ]:

def function_result_02():


In [ ]:
    print('[plot examples of the training segmentation labels]')
    print('') 


In [ ]:
    nRow = 8
    nCol = 6
    index_data  = np.arange(0, nRow * nCol)
    _,label_train = dataset_train[index_data]
    label_train = label_train[0]


In [ ]:
    plot_data_grid(label_train, index_data, nRow, nCol)


In [ ]:

# In[35]:


In [ ]:

def function_result_03():


In [ ]:
    print('[plot examples of the training segmentation results]')
    print('') 


In [ ]:
    nRow = 8
    nCol = 6
    index_data          = np.arange(0, nRow * nCol)
    image_train,_       = dataset_train[index_data] 
    image_train         = image_train[0].unsqueeze(dim=1).to(device)
    prediction_train    = compute_prediction(model, image_train)


In [ ]:
    plot_data_tensor_grid(prediction_train, index_data, nRow, nCol)


In [ ]:

# In[36]:


In [ ]:

def function_result_04():


In [ ]:
    print('[plot examples of the testing images]')
    print('') 


In [ ]:
    nRow = 8
    nCol = 6
    index_data      = np.arange(0, nRow * nCol)
    image_test,_    = dataset_test[index_data]
    image_test      = image_test[0]


In [ ]:
    plot_data_grid(image_test, index_data, nRow, nCol)


In [ ]:

# In[37]:


In [ ]:

def function_result_05():


In [ ]:
    print('[plot examples of the testing segmentation labels]')
    print('') 


In [ ]:
    nRow = 8
    nCol = 6
    index_data = np.arange(0, nRow * nCol)
    _,label_test = dataset_test[index_data]
    label_test = label_test[0]


In [ ]:
    plot_data_grid(label_test, index_data, nRow, nCol)


In [ ]:

# In[38]:


In [ ]:

def function_result_06():


In [ ]:
    print('[plot examples of the testing segmentation results]')
    print('') 


In [ ]:
    nRow = 8
    nCol = 6
    index_data      = np.arange(0, nRow * nCol)
    image_test,_    = dataset_test[index_data]
    image_test      = image_test[0].unsqueeze(dim=1).to(device)
    prediction_test = compute_prediction(model, image_test)


In [ ]:
    plot_data_tensor_grid(prediction_test, index_data, nRow, nCol)


In [ ]:

# In[39]:


In [ ]:

def function_result_07():


In [ ]:
    print('[plot the training loss]')
    print('') 


In [ ]:
    plot_curve_error(loss_mean_train, loss_std_train, 'epoch', 'loss', 'loss (training)')


In [ ]:

# In[40]:


In [ ]:

def function_result_08():


In [ ]:
    print('[plot the training accuracy]')
    print('') 


In [ ]:
    plot_curve_error(accuracy_mean_train, accuracy_std_train, 'epoch', 'accuracy', 'accuracy (training)')


In [ ]:

# In[41]:


In [ ]:

def function_result_09():


In [ ]:
    print('[plot the testing loss]')
    print('') 


In [ ]:
    plot_curve_error(loss_mean_test, loss_std_test, 'epoch', 'loss', 'loss (testing)')


In [ ]:

# In[42]:


In [ ]:

def function_result_10():


In [ ]:
    print('[plot the testing accuracy]') 
    print('') 


In [ ]:
    plot_curve_error(accuracy_mean_test, accuracy_std_test, 'epoch', 'accuracy', 'accuracy (testing)')


In [ ]:

# In[43]:


In [ ]:

def function_result_11():


In [ ]:
    print('[print the training loss at the last 10 epochs]')
    print('') 


In [ ]:
    data_last = get_data_last(loss_mean_train, -10)
    index = np.arange(0, 10)
    print_curve(data_last, index)


In [ ]:

# In[44]:


In [ ]:

def function_result_12():


In [ ]:
    print('[print the training accuracy at the last 10 epochs]')
    print('') 


In [ ]:
    data_last = get_data_last(accuracy_mean_train, -10)
    index = np.arange(0, 10)
    print_curve(data_last, index)


In [ ]:

# In[45]:


In [ ]:

def function_result_13():


In [ ]:
    print('[print the testing loss at the last 10 epochs]')
    print('') 


In [ ]:
    data_last = get_data_last(loss_mean_test, -10)
    index = np.arange(0, 10)
    print_curve(data_last, index)


In [ ]:

# In[46]:


In [ ]:

def function_result_14():


In [ ]:
    print('[print the testing accuracy at the last 10 epochs]')
    print('') 


In [ ]:
    data_last = get_data_last(accuracy_mean_test, -10)
    index = np.arange(0, 10)
    print_curve(data_last, index)


In [ ]:

# In[47]:


In [ ]:

def function_result_15():


In [ ]:
    print('[print the best training accuracy within the last 10 epochs]')
    print('') 


In [ ]:
    value = get_max_last_range(accuracy_mean_train, -10)
    print('best training accuracy = %12.10f' % (value))


In [ ]:

# In[48]:


In [ ]:

def function_result_16():


In [ ]:
    print('[print the best testing accuracy within the last 10 epochs]')
    print('') 


In [ ]:
    value = get_max_last_range(accuracy_mean_test, -10)
    print('best testing accuracy = %12.10f' % (value))


In [ ]:

# ---


In [ ]:
# # RESULTS


In [ ]:
# ---


In [ ]:
# In[49]:


In [ ]:

number_result = 16


In [ ]:
for i in range(number_result):


In [ ]:
    title           = '# RESULT # {:02d}'.format(i+1) 
    name_function   = 'function_result_{:02d}()'.format(i+1)


In [ ]:
    print('') 
    print('################################################################################')
    print('#') 
    print(title)
    print('#') 
    print('################################################################################')
    print('') 


In [ ]:
    eval(name_function)


In [ ]:

# In[ ]:
